## Torch下的GPU支持

### 1. 支持GPU的张量库


In [ ]:
require 'cutorch'
print(  cutorch.getDeviceProperties(cutorch.getDevice()) )

-- 我们默认使用Float，单精度计算
torch.setdefaulttensortype('torch.FloatTensor')

Torch支持tensor在CPU/GPU各种的切换，cutorch支持在GPU环境下执行Tensor的各种操作。

In [ ]:
-- 在GPU内进行计算
t1 = torch.CudaTensor(100):fill(0.5)
t2 = torch.CudaTensor(100):fill(1)
t1:add(t2)

-- GPU和CPU内存切换
t1_cpu = t1:float()
t1:zero()
t1[{}] = t1_cpu  -- copies the data back to the GPU, with no new alloc
t1_new = t1_cpu:cuda()  -- allocates a new tensor

### 2. 支持GPU执行环境的神经网络库

Torch包含一个GPU执行的神经网络库，这个库和Tensor库的GPU支持一样，接口与GPU版本一样。
同样神经网络模型支持CPU与GPU之间的无缝切换。